In [1]:
import geopandas as gpd
import pandas as pd
#import parquet

In [2]:
wells = pd.read_csv("raw_data/cleaned_well_data.csv")

# wells = pd.read_csv("raw_data/Well_Registry.csv")
wells['date'] = pd.to_datetime(wells['date']) 
wells.info()

C:\Users\liamw\AppData\Local\Temp\ipykernel_20948\1296714404.py:1: DtypeWarning: Columns (36,45) have mixed types. Specify dtype option on import or set low_memory=False.
  wells = pd.read_csv("raw_data/cleaned_well_data.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289301 entries, 0 to 289300
Data columns (total 56 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Unnamed: 0                  289301 non-null  int64         
 1   wellid                      289301 non-null  int64         
 2   WLWA_ID                     289301 non-null  int64         
 3   date                        288969 non-null  datetime64[ns]
 4   depth                       275285 non-null  float64       
 5   WLWA_WATER_LEVEL_ELEVATION  275285 non-null  float64       
 6   WLWA_SOURCE_CODE            283252 non-null  object        
 7   WLWA_METHOD_CODE            272560 non-null  object        
 8   WLWA_REMARK_CODE            24846 non-null   object        
 9   SITE_WELL_ALTITUDE          289301 non-null  float64       
 10  SITE_LOCAL_ID               289301 non-null  object        
 11  SITE_LATIT_DEGREE           289301 non-

In [3]:
wells = wells[wells["date"]>="1/1/2000"]
wells = wells[wells["date"]<="1/1/2020"]
wells["utc_seconds"] = pd.to_datetime(wells['date'].astype(int)).astype(int)/ 10**9

TypeError: Converting from datetime64[ns] to int32 is not supported. Do obj.astype('int64').astype(dtype) instead

In [ ]:
wells = wells[wells["depth"].notnull()]
wells.info()

In [ ]:
well1 = wells[wells["wellid"]==343957112322001]
well1

In [ ]:
import numpy as np
well1
df_unix_sec = pd.to_datetime(well1['date']).astype(int)/ 10**9
f = np.polyfit(well1["utc_seconds"], well1['depth'], deg=1)

# Slope
f[0]*(60*60*24*365*20)

In [ ]:
def calculate_trend_slope(df):
    f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
    # Slope
    return f[0]*(60*60*24*365*20)


In [ ]:
wells = wells.groupby('wellid').agg({
    'SITE_LONGIT_DECIMAL':'first',
    'SITE_LATITUDE_DECIMAL':'first',
    'depth': lambda x: list(x),
    'utc_seconds': lambda x: list(x)
    }).reset_index()

In [ ]:

wells["number_of_measurements"] = wells.apply(lambda well: len(well["depth"]), axis=1)
wells = wells[wells["number_of_measurements"]>=5]
wells["depth_20_year_trend"] = wells.apply(calculate_trend_slope, axis=1)
wells.info()

In [ ]:
wells[wells["wellid"]==343957112322001]

In [ ]:
wells.to_parquet("our_data/wells_with_depth_trend.parquet")